# UMAP for bird-songs  
---

In [ ]:
## Installs for Coursera ^^
# conda install -c conda-forge librosa umap-learn
# conda install -c plotly plotly=4.14.3

In [1]:
## Installs for CoLab
!pip install umap-learn

     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 1.2MB 19.8MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=fbc68a86a9388a4a2bc8709af37549047e22497a99a349b7d8e2a7c83f8fe883
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=479a30bda0c2a7a89a72577625b88191f762851b3eebe97be8ec96ec929cb3cd
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent


In [2]:
# For Colab only
# 4/1AY0e-g6CTocvBK45PWlJu2ycBuOMTgu36b-VZgpnCXitW_Vy3ckpGGBzur8
from google.colab import drive
drive.mount('content')
# /content/content/MyDrive/bird-songs/audio

Mounted at content


In [3]:
from umap import UMAP
from sklearn.datasets import load_digits
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

import os
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import pandas as pd
import glob
# import ffmpeg
%matplotlib inline

# Set general font size
plt.rcParams['font.size'] = '14'

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm


In [16]:
# Check whether google drive mounted
path = '/content/content/MyDrive/bird-songs/'
if os.path.isdir(path) == True:
    print('Google Drive Mounted')
    run_on_colab = True
else:
    print('Using local drive')
    run_on_colab = False
    

Google Drive Mounted


In [18]:
def get_audio_filenames(audio_folder):
    '''Create a list of audio files in the provided folder'''
    
    audio_files = []
    for fn_mp3 in glob.glob(f"{audio_folder}*"):
        audio_files.append(fn_mp3)
    return audio_files

if run_on_colab ==True:
    audio_filenames = get_audio_filenames('/content/content/MyDrive/bird-songs/audio/')
    print(f'{len(audio_filenames)} audio files appended to list from Google Drive')
else:
    audio_filenames = get_audio_filenames('./')
    print(f'{len(audio_filenames)} audio files appended to list from local drive')


500 audio files appended to list from Google Drive


In [15]:
def create_df_xeno_canto(csv_file, audio_folder, min_dur, max_dur):
    '''Create pandas dataframe with features provided by xeno-canto and append file name of audio file'''
    
    df = pd.read_csv(csv_file, header=None)
    print(df.head())
    df.columns = ['id', 'gen', 'sp', 'ssp', 'en', 'cnt', 'loc','type', 'q', 'length', 'bird-seen', 'link']
    if run_on_colab ==True:
        df['filename'] = df['id'].map('/content/content/MyDrive/bird-songs/audio/{}.mp3'.format, na_action='ignore')
    else:
        df['filename'] = df['id'].map('audio/{}.mp3'.format, na_action='ignore')
    
    df['seconds'] = df['length'].apply(lambda x: int(x[0])*60 + int(x[-2:]))
    
    df = df[(df['seconds']>=min_dur) & (df['seconds']<=max_dur)]
    
    df.set_index('id', inplace=True)

    return df


if run_on_colab ==True:
    df_xeno_canto = create_df_xeno_canto('/content/content/MyDrive/bird-songs/features.csv', 'audio/', 2, 10)
else:
    df_xeno_canto = create_df_xeno_canto('features.csv', 'audio/', 2, 10)


# df_xeno_canto.sort_values(by='seconds', ascending=True).head()
df_xeno_canto

       0            1   ...   10                                    11
0  527379  Dendrocygna  ...  yes  //www.xeno-canto.org/527379/download
1  480387  Dendrocygna  ...  yes  //www.xeno-canto.org/480387/download
2  316537  Dendrocygna  ...  yes  //www.xeno-canto.org/316537/download
3  192058  Dendrocygna  ...  yes  //www.xeno-canto.org/192058/download
4  172910  Dendrocygna  ...  yes  //www.xeno-canto.org/172910/download

[5 rows x 12 columns]


,gen,sp,ssp,en,cnt,loc,type,q,length,bird-seen,link,filename,seconds
id,,,,,,,,,,,,,
316537,Dendrocygna,autumnalis,NaN,Black-bellied Whistling Duck,United States,"Chalkley Road, Cameron Parish, Louisiana",call,A,0:07,yes,//www.xeno-canto.org/316537/download,/content/content/MyDrive/bird-songs/audio/3165...,7
452052,Dendrocygna,bicolor,NaN,Fulvous Whistling Duck,United States,"Lake Apopka Wildlife Drive, Orange County, Flo...",flight call,A,0:06,yes,//www.xeno-canto.org/452052/download,/content/content/MyDrive/bird-songs/audio/4520...,6
147330,Dendrocygna,bicolor,NaN,Fulvous Whistling Duck,United States,"Santa Ana National Wildlife Refuge, Texas",call,A,0:09,yes,//www.xeno-canto.org/147330/download,/content/content/MyDrive/bird-songs/audio/1473...,9
222338,Dendrocygna,bicolor,NaN,Fulvous Whistling Duck,United States,"Anahuac NWR, TX",call,B,0:06,unknown,//www.xeno-canto.org/222338/download,/content/content/MyDrive/bird-songs/audio/2223...,6
147331,Dendrocygna,bicolor,NaN,Fulvous Whistling Duck,United States,"Santa Ana National Wildlife Refuge, Texas",call,B,0:06,yes,//www.xeno-canto.org/147331/download,/content/content/MyDrive/bird-songs/audio/1473...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155583,Mystery,mystery,NaN,Identity unknown,Canada,"Wood Buffalo (near Fort McMurray), Division N...",call,B,0:04,no,//www.xeno-canto.org/155583/download,/content/content/MyDrive/bird-songs/audio/1555...,4
152879,Mystery,mystery,NaN,Identity unknown,Canada,"Wood Buffalo (near Fort McMurray), Division N...",call,B,0:09,no,//www.xeno-canto.org/152879/download,/content/content/MyDrive/bird-songs/audio/1528...,9
152774,Mystery,mystery,NaN,Identity unknown,Canada,"Wood Buffalo (near Fort McMurray), Division N...",call,B,0:09,no,//www.xeno-canto.org/152774/download,/content/content/MyDrive/bird-songs/audio/1527...,9


In [ ]:
def create_mfcc_spectral_features(number_mfcc, sample_rate=22050):
    ''' '''


    # Create df to hold mfcc spectral features
    df = pd.DataFrame(columns = ['mfcc_avg' + str(item+1) for item in list(range(number_mfcc))] +
                                ['mfcc_std' + str(item+1) for item in list(range(number_mfcc))], 
                      index = df_xeno_canto['filename'])
        
    display(df.head())
    for audio_filename in tqdm(df_xeno_canto['filename']):


        y, sr = librosa.load(audio_filename, sr=sample_rate, mono=True)
        mfccs = librosa.feature.mfcc(y=y[:110250], sr=sr, n_mfcc=number_mfcc, hop_length=2048)
        # print(mfccs.shape)

        mfcc_means = [np.mean(item) for item in mfccs]
        mfcc_stds = [np.std(item) for item in mfccs]
            # print("mfcc means:", mfcc_means, "\nmfcc std deviations:", mfcc_stds)
        df.loc[audio_filename] = mfcc_means + mfcc_stds
  
    #print(mfcc_stds)
    
    return df


number_mfcc = 12
df_mfcc = create_mfcc_spectral_features(number_mfcc)


,mfcc_avg1,mfcc_avg2,mfcc_avg3,mfcc_avg4,mfcc_avg5,mfcc_avg6,mfcc_avg7,mfcc_avg8,mfcc_avg9,mfcc_avg10,mfcc_avg11,mfcc_avg12,mfcc_std1,mfcc_std2,mfcc_std3,mfcc_std4,mfcc_std5,mfcc_std6,mfcc_std7,mfcc_std8,mfcc_std9,mfcc_std10,mfcc_std11,mfcc_std12
filename,,,,,,,,,,,,,,,,,,,,,,,,
/content/content/MyDrive/bird-songs/audio/316537.mp3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/content/content/MyDrive/bird-songs/audio/452052.mp3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/content/content/MyDrive/bird-songs/audio/147330.mp3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/content/content/MyDrive/bird-songs/audio/222338.mp3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/content/content/MyDrive/bird-songs/audio/147331.mp3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


FileNotFoundError: ignored

In [ ]:
def create_mel_spectral_features(sample_rate=22050):
    ''' '''

    # Create df to hold mfcc spectral features
    df = pd.DataFrame(columns = ['mel_avg' + str(item+1) for item in list(range(128))] +
                                ['mel_std' + str(item+1) for item in list(range(128))], 
                      index = df_xeno_canto['filename'])
        

    for audio_filename in tqdm(df_xeno_canto['filename']):
        

        y, sr = librosa.load(audio_filename, sr=sample_rate, mono=True)
        mels = librosa.feature.melspectrogram(y=y[:110250], sr=sample_rate, n_fft=2048, hop_length=512)
        # print(mels.shape)
        mels_means = [np.mean(item) for item in mels]
        mels_stds = [np.std(item) for item in mels]
        # print("mel means:", mels_means, "\nmel std deviations:", mels_stds)
        df.loc[audio_filename] = mels_means + mels_stds
  
    return df



df_mel = create_mel_spectral_features()

# df_mel.head()

In [ ]:
def create_df_umap(df1, df2, df3):

    df_umap_step1 = df1.merge(df2, how ='left', left_on='filename', right_index=True)
    df_umap_step2 = df_umap_step1.merge(df3, how ='left', left_on='filename', right_index=True)
    return df_umap_step2


df_umap = create_df_umap(df_xeno_canto, df_mfcc, df_mel)
df_umap.head()

In [ ]:
def create_umap_chart(df, num_features=20, color_feature='species'):
    
    umap_2d = UMAP(random_state=0)
    umap_2d.fit(df.iloc[:, -num_features:].values)

    projections = umap_2d.transform(df.iloc[:, -num_features:].values)

    fig = px.scatter(
        projections, x=0, y=1,
        color=list(df[color_feature].astype(str)), labels={'color': color_feature},
        title = f"UMAP Plot of bird audio samples colored by {color_feature}"
    )
    
    return fig

num_mfcc_features = number_mfcc*2
# create_umap_chart(df_umap, num_mfcc_features, 'genus')

num_mel_features = df_mel.shape[1]
# create_umap_chart(df_umap, num_mel_features, 'genus')

num_mfcc_and_mel_features = num_mfcc_features + num_mel_features
print(f"Total number of features is {num_mfcc_and_mel_features}")
print(f"Total number of bird audio samples is {len(df_umap)}")
create_umap_chart(df_umap, num_mfcc_features+num_mel_features, 'genus')